# <span style="color:green">Plancton Images Classifier</span>
*von Bourges Julian, Hahn Sandro, Schmalzl Maximilian*

Definition: 
Plankton ist die Bezeichnung für die Gesamtheit der Organismen, die im Wasser von Meeren, Flüssen und Seen leben. Sie bewegen sich nicht oder nur sehr wenig aus eigener Kraft, weshalb deren Schwimmrichtung von der Strömung vorgegeben wird.

## <span style="color:rgb(166, 212, 46)">I. Datenvorbereitung</span>

 




Unser Datensatz "Converted" besteht aus 219 063 .png Dateien. Jede einzelne .png bildet ein Plankton ab. Um mit diesem Datensatz arbeiten zu können, müssen die einzelnen Daten, also die .png Bilder, zunächst aufbereitet und skaliert werden. <br> 
Da Java-Code direkt in Maschinencode kompiliert wird, ist er in der Regel schneller. Unsere Daten haben wir deshalb zunächst mit Java genauer betrachtet und aufbereitet. 

Um einen Eindruck von der Größe der Bilder zu bekommen haben wir mit der getPercentages()-Funktion die Anzahl der Pixel berechnet, in die die einzelnen Bilder jeweils in Höhe und Breite skaliert sind. Anschließend haben wir die Bilder mit 0 - 99, 100 - 199, ... und 1300 - 1399 Pixeln zu Gruppen zusammengefast und uns die jeweilige Repräsentation der Gruppe in Prozent ausgeben lassen.

In [ ]:
public static void getPercentages() {

        DecimalFormat df = new DecimalFormat("0.00");
        int sum = 0;

        for(int entry : WIDTHS_ARR) {
            sum += entry;
        }

        System.out.println("Widths:");

        for(int i = 0; i < WIDTHS_ARR.length; i++) {
            double percent = ((double) WIDTHS_ARR[i] / sum) * 100.0D;
            System.out.println(i + ": " + WIDTHS_ARR[i] + " (" + df.format(percent) + "%)");
        }

        System.out.println();
        System.out.println("Heights:");

        for(int i = 0; i < HEIGHTS_ARR.length; i++) {
            double percent = ((double) HEIGHTS_ARR[i] / sum) * 100.0D;
            System.out.println(i + ": " + HEIGHTS_ARR[i] + " (" + df.format(percent) + "%)");
        }

    }



Die Funktion erzeugt folgenden Output (ersichtlicher dargestellt in einer Tabelle):

<table>
  <tr>
    <th>Widths</th>
    <th>Counts</th>
    <th></th>
    <th>Heights</th>
    <th>Counts</th>
  </tr>
  <tr>
    <td>0</td>
    <td>21592 (9,86%)</td>
    <td></td>
    <td>0</td>
    <td>19926 (9,10%)</td>
  </tr>
  <tr>
    <td>1</td>
    <td>72812 (33,24%)</td>
    <td></td>
    <td>1</td>
    <td>60609 (27,67%)</td>
  </tr>
  <tr>
    <td>2</td>
    <td>93711 (42,78%)</td>
    <td></td>
    <td>2</td>
    <td>99519 (45,43%)</td>
  </tr>
  <tr>
    <td>3</td>
    <td>15954 (7,28%)</td>
    <td></td>
    <td>3</td>
    <td>19706 (9,00%)</td>
  </tr>
  <tr>
    <td>4</td>
    <td>7324 (3,34%)</td>
    <td></td>
    <td>4</td>
    <td>9451 (4,31%)</td>
  </tr>
  <tr>
    <td>5</td>
    <td>3609 (1,65%)</td>
    <td></td>
    <td>5</td>
    <td>4675 (2,13%)</td>
  </tr>
  <tr>
    <td>6</td>
    <td>1838 (0,84%)</td>
    <td></td>
    <td>6</td>
    <td>2415 (1,10%)</td>
  </tr>
  <tr>
    <td>7</td>
    <td>973 (0,44%)</td>
    <td></td>
    <td>7</td>
    <td>1297 (0,59%)</td>
  </tr>
  <tr>
    <td>8</td>
    <td>558 (0,25%)</td>
    <td></td>
    <td>8</td>
    <td>692 (0,32%)</td>
  </tr>
  <tr>
    <td>9</td>
    <td>317 (0,14%)</td>
    <td></td>
    <td>9</td>
    <td>380 (0,17%)</td>
  </tr>
  <tr>
    <td>10</td>
    <td>213 (0,10%)</td>
    <td></td>
    <td>10</td>
    <td>287 (0,13%)</td>
  </tr>
  <tr>
    <td>11</td>
    <td>74 (0,03%)</td>
    <td></td>
    <td>11</td>
    <td>47 (0,02%)</td>
  </tr>
  <tr>
    <td>12</td>
    <td>34 (0,02%)</td>
    <td></td>
    <td>12</td>
    <td>20 (0,01%)</td>
  </tr>
  <tr>
    <td>13</td>
    <td>53 (0,02%)</td>
    <td></td>
    <td>13</td>
    <td>38 (0,02%)</td>
  </tr>
</table>


Wir sehen, dass, sowohl in Breite und Höhe, die meist repräsentierte Gruppe 200 - 299 Pixel groß ist. Gedicht gefolgt von der Gruppe mit 100 - 199 Pixeln. Alle anderen Gruppen beinhalten relativ wenige Bilder.

Damit können wir folgendes machen:

In [ ]:
public static void resizeAll() throws Exception {

    final String inputRootPath = "D://KI_Plankton//Plankton_Converted//Converted";
    final String outputRootPath = "D://KI_Plankton//Output200//";
    final int targetWidth = 200;
    final int targetHeight = 200;

    File rootDir = new File(inputRootPath);
    int i = 0;

    for(String pictureName : rootDir.list()) {

        File input = new File(inputRootPath + "//" + pictureName);
        File output = new File(outputRootPath + pictureName);
        resizeImage(targetWidth, targetHeight, input, output);

        if(++i % 1_000 == 0) {
            System.out.println("processed " + i + " pictures");
        }

    }

}

public static void resizeImage(int targetWidth, int targetHeight, File input, File output) throws Exception {

    BufferedImage bufferedImage = ImageIO.read(input);

    boolean fitHeight = bufferedImage.getHeight() > bufferedImage.getWidth();
    Scalr.Mode scaleMode = fitHeight ? Scalr.Mode.FIT_TO_HEIGHT : Scalr.Mode.FIT_TO_WIDTH;

    BufferedImage scaledImage = resizeImage(bufferedImage, targetWidth, targetHeight, scaleMode);

    int y_val = (int) ((targetHeight/2.0D) - (scaledImage.getHeight()/2.0D));
    int x_val = (int) ((targetWidth/2.0D) - (scaledImage.getWidth()/2.0D));

    BufferedImage newImage = new BufferedImage(targetWidth, targetHeight, BufferedImage.TYPE_INT_RGB);
    Graphics2D graphics2D = newImage.createGraphics();
    graphics2D.setPaint(new Color(0, 0, 0));
    graphics2D.fillRect(0, 0, newImage.getWidth(), newImage.getHeight());

    if(fitHeight) {
        graphics2D.drawImage(scaledImage, x_val, 0, null);
    } else {
        graphics2D.drawImage(scaledImage, 0, y_val, null);
    }

    graphics2D.dispose();
    ImageIO.write(newImage, "png" , output);

}

public static BufferedImage resizeImage(BufferedImage originalImage, int targetWidth, int targetHeight, Scalr.Mode mode) throws Exception {
    Scalr.Method method = Scalr.Method.QUALITY;
    return Scalr.resize(originalImage, method, mode, targetWidth, targetHeight, Scalr.OP_ANTIALIAS);
}

## II. Decision Trees

## III. Feedforward Neural Network

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# Beispiel: Anzahl der Klassen (Anpassen Sie dies entsprechend Ihren Daten)
num_classes = 10

# Erstellen Sie ein sequentielles Modell
model = Sequential()

# Fügen Sie Schichten zum Modell hinzu
model.add(Dense(128, input_shape=(input_size,), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Kompilieren Sie das Modell
model.compile(loss='categorical_crossentropy',  # Anpassen Sie dies entsprechend Ihrem Problem
              optimizer='adam',
              metrics=['accuracy'])

# Zeigen Sie eine Zusammenfassung des Modells an
model.summary()


## IV. Covolutional Neural Network

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Beispiel: Bildgröße und Anzahl der Klassen (Anpassen Sie dies entsprechend Ihren Daten)
img_size = (64, 64, 3)  # Hier angenommen, dass die Bilder 64x64 Pixel mit 3 Farbkanälen sind
num_classes = 10

# Erstellen Sie ein sequentielles Modell
model = Sequential()

# Fügen Sie Convolutional- und Pooling-Schichten hinzu
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=img_size))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flach machen (flatten) Sie die Ausgabe für den Fully Connected Teil
model.add(Flatten())

# Fügen Sie Dense-Schichten für die Klassifikation hinzu
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Kompilieren Sie das Modell
model.compile(loss='categorical_crossentropy',  # Anpassen Sie dies entsprechend Ihrem Problem
              optimizer='adam',
              metrics=['accuracy'])

# Zeigen Sie eine Zusammenfassung des Modells an
model.summary()
